# Introduction: What is Julia

Julia is a high-level, high-performance, dynamic programming language well-suited for high-performance numerical analysis and computational science. Or, as stated by its makers; [..] each [programming language] is perfect for some aspects of the work and terrible for others. Each one is a trade-off.

>**We are greedy: we want more.**
>We want a language that's open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a language that's homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled.

- Made in 2009 at MIT by Jeff Bezanson, Stefan Karpinski, Viral B. Shah, and Alan Edelman.
- First released in 2012, v1.0 release in 2018. Currently at v1.9.3 with 1.10 around the corner (beta).
- Open MIT license, developed on Github, currently at 55k commits, 1425 contributors and 43k stars.
- Only 11 years old (!)

## Solving the two language problem
Normally you have two languages.
- A language to prototype, easy but slow (Python, R, Matlab)
- A language for production, hard but fast (C, C++, Fortran)

This happens everywhere, and you might not notice it at first. Most of our (geospatial) packages depend on C++ libraries, like GDAL, GEOS and PROJ. Packages like numpy and pytorch in Python use a lot of C(++) under the hood.

Julia is used by many companies around the world. Noticable uses include financial analysis by investors/banks. Of interest to us is the [Celeste](https://github.com/jeff-regier/Celeste.jl) project in 2017 where Julia was used or astronomical analysis to achieve 1.54 petaFLOPS using 1.3 million threads on a supercomputer. And the 
Climate Modeling Alliance (CLIMA) who use Julia for their next-gen global climate model [Oceananigans.jl](https://github.com/CliMA/Oceananigans.jl.).

At Deltares, Julia was first tried out when working on large scale pointclouds by Martijn Visser. Python was too slow, but C++ was a step too far. The language was still in its infancy, so there were some issues, but it generally worked well and enabled us to process the billions of points for our project in Indonesia. The used and generated packages are now part of JuliaGeo: https://juliageo.org/, with geospatial libraries linked such as GDAL, GEOS, Proj etc.

---
## Features (the good)

### Multiple dispatch
Julia is a dynamically typed language, like Matlab or Python, so it will try to figure out the correct types on the fly. Specifying types is optional, and can be used for instance so select the right method based on input types.

In [16]:
"""
    power(a, b)

Generic `power`.
"""
function power(a, b)
    a^b
end

"""
    power(a::Integer, b::Integer)

Special case for `power` handling of Integers.
"""
function power(a::Integer, b::Integer)
    # special sauce here
    a^b::Integer  # type isn't required here as it is inferred
end

power

In [17]:
?power

search: power powermod



```
power(a, b)
```

Generic `power`.

---

```
power(a::Integer, b::Integer)
```

Special case for `power` handling of Integers.


---
User-defined types are as fast and compact as built-ins. This means that creating your own types is fully supported and with a few lines can be  understood by the ecosystem. This example also makes use of multiple dispatch, to which I come back later.

In [5]:
struct MyMeasurement
    val::Float64
    err::Float64
end
Base.:*(m::MyMeasurement, other::Number) = MyMeasurement(m.val * other, m.err * other)
MyMeasurement(1.0, 0.1) * 2.0

MyMeasurement(2.0, 0.2)

---
### Interopability
Call C or Fortran functions directly (no wrappers or special APIs needed). Many of the types are identical, so they can be load/cast without copies. Python interopability is also supported with PyCall.jl, and R with RCall.jl.

In [6]:
ccall(:clock, Int32, ())

8969032

---
### Unicode
Efficient support for Unicode, including but not limited to UTF-8. This makes it easier to have code that looks similar to the equations as you may read them in a paper.

In [7]:
# use actual example
using Statistics
values = [1,2,3,4,5]
√(sum((values .- mean(values)).^2) / (length(values) - 1))

println(π * 1.0)

🎲() = rand(1:6)  # you can go too far... 😜
🎲()
ρ = 1.0

3.141592653589793


1.0

---
### Loops
No need to vectorize code for performance; devectorized code is fast. Just write out your ```for``` loops.

In [18]:
# Euler–Riemann zeta function

function ζ()
    total = 0.0
    for k = 1:100_000_000
        total += 1.0/(k*k)
    end
    sqrt(total*6)
end
@time @info ζ()  # note that you want to precompile functions and you need a proper benchmark, please use BenchmarkTools

# For completeness, the vectorized notation
function ζv()
    a = 1:100_000_000
    total = sum(1 ./ (a.*a))
    sqrt(total*6)
end
@time @info ζv()

a = [1,2,3]

  0.128080 seconds (117 allocations: 9.305 KiB)


[ Info: 3.14159264498239


  0.123337 seconds (356 allocations: 762.969 MiB, 2.62% gc time)


[ Info: 3.1415926440404984


3-element Vector{Int64}:
 1
 2
 3

---
### Arrays
Full support for Multi-Dimensional Arrays. Unlike Python, which needs Numpy for arrays (itself built in C btw), Julia support multi-dimensional arrays out of the box.

In [9]:
A = rand(5)  # random
B = ones(5,2,3)  # 5x2x3 array of 1s
C = Matrix(undef, 5, 5)  # initialize without (re)setting memory values

[1,2,3,4,5] .* [1,2,3,4,5]

5-element Vector{Int64}:
  1
  4
  9
 16
 25

---
### Metaprogramming
Lisp-like macros and other metaprogramming facilities. Julia represents its own code as a data structure of the language itself. This enables macros, but in the end *anything* you can think of. We've used it to generate thousands of lines of code that would normally have to be handwritten. It's really programming your programming.


In [10]:
# Everything's a string first
prog = "1 + 1"
ex1 = Meta.parse(prog)
println("$(typeof(ex1)) $ex1")

# An expression has a head and args
println(ex1.head)
println(ex1.args)

# You can create this yourself as well
ex2 = Expr(:call, :+, 1, 1)
ex2 == ex1

Expr 1 + 1
call
Any[:+, 1, 1]


true

---
### Package manager
A built in package manager. This prevents the need for many distributions and or tools such as anaconda and pip.

In [11]:
]?

  Welcome to the Pkg REPL-mode. To return to the julia> prompt, either press
  backspace when the input line is empty or press Ctrl+C.

  Full documentation available at https://pkgdocs.julialang.org/

  Synopsis

  pkg> cmd [opts] [args]

  Multiple commands can be given on the same line by interleaving a ; between
  the commands. Some commands have an alias, indicated below.

  Commands

  activate: set the primary environment the package manager manipulates

  add: add packages to project

  build: run the build script for packages

  compat: edit compat entries in the current Project and re-resolve

  develop, dev: clone the full package repo locally for development

  free: undoes a pin, develop, or stops tracking a repo

  gc: garbage collect packages not used for a significant time

  generate: generate files for a new project

  help, ?: show this message

  instantiate: downloads all the dependencies for the project

  pin: pins the version of packages

  precompile: precompil

## When to use
This lecture aims at giving you a good introduction to Julia and to understand when you could choose to use it. We do not propose you dump Python, Matlab, Fortran or C++ now (well, maybe Matlab). As described above, we think the clear use case for Julia is technical computing. Domain specific programs that require High Performance Computing (HPC).

We still use Python a lot for simple scripts or webservices and advise you to keep doing that. However, if you hit the limits of what you can do with Python or R, you might want to try Julia.

### The bad
The main drawback of Julia is the young ecosystem, while there are multiple state of the art libraries, mainly focused on computing, the ecosystem is lacking for more general programming, compared to a language like Python.

### The ugly
- Can't compile code to a small executable (~500MB)
- Compilation times or Time to first plot (TTFP)

## Excercise
Please try to make a function that:
- prints "Hello World"
- takes a `name` argument and prints "Hello Maarten" if the given name is "Maarten"
- takes a `name` argument but prints "Hello World" if the give name is empty

See the manual to help you out https://docs.julialang.org/en/v1/manual/functions/

In [12]:
# please write your function here. Shift enter or the play button above executes it